# Power Factory 

Group E

Project 3 Solution IPYNB

## Required Libraries and Setup

In [1]:
import sys
sys.path.append('C:/Program Files/DIgSILENT/PowerFactory 15.2/Python/3.4')

In [2]:
import numpy as np
import powerfactory as pf
import pandas as pd
import datetime as dt
import os
import PythonMagick
from IPython.display import Image
import time
app = pf.GetApplication()
app.ActivateProject('LabPowerFactory')

0

## Exercise 2

## Necessary Classes

In [3]:
# Load flow calculation class
load_flow_class = app.GetFromStudyCase('ComLdf')
# Time_object
study_time = app.GetFromStudyCase('SetTime')

## Get Objects

In [4]:
subs=app.GetCalcRelevantObjects('*.ElmTrfstat')
lines=app.GetCalcRelevantObjects('*.ElmLne')
feeders=app.GetCalcRelevantObjects('*.ElmFeeder')

## Example
This is the code which should be executed after each time we change the configuration

In [5]:

load_flow_class.Execute()

0

## Exercise 3

### Example Code

Since there is no duplicates in the result set, we conclude that this network is radial. 

In [ ]:
app.GetFromStudyCase('ComLdf').Execute()
layers = {}
net_layout = sorted(set([sub.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') for sub in subs]))
for l in net_layout:
    print(l)
    layers[l] = []
    for s in subs: 
        if s.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') == l:
            layers[l].append(s.loc_name)
layers



In [31]:
#print(subs[2].loc_name)
#print(subs[2].GetAttribute('e:pBusbar'))
#subs[2].GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot')
for sub in subs:
    print(sub)

<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(1).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(3).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(4).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(7).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(9).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(11).ElmTrfstat</l3>
<l3>\xoxgur.IntUser\LabPowerFactory.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Trafo Substation(14).ElmTrf

## Build Incidence Matrix
Here we are building the instance matrix manually.

The convention from the supplied example image is used:
 - Branch number is the number of the corresponding node, which has the incoming connection. For example, 1 -> 2 means, 1 connects to 2, so the branch number is 2.
 
Size of the matrix is 1 + 5 + 17 = 23. 
 - 1 stands for the root 63 kV station
 - 5 for 20 kV substations
 - 17 for the trafo substations.

Here, we are configuring matrix as follows, first column is for the root node, next 5 columns are for the 20 kV substations and last 17 columns are for trafo substations.  
So when we are refering the first trafo substation, corresponding index is calculated as follows:

1(root) + 5(20 kV stations) + 1(index we want) - 1(indexing starts from 0)


#### Indexes for the Trafo substations
We need this correction, since ids of substations have gaps, so they are aligned for more compact incidence matrix.

Trafo Substation(1) -> 1

Trafo Substation(2) -> 2

Trafo Substation(3) -> 3

Trafo Substation(4) -> 4

Trafo Substation(7) -> 5

Trafo Substation(9) -> 6

Trafo Substation(11)-> 7

Trafo Substation(14)-> 8

Trafo Substation(16)-> 9

Trafo Substation(18)-> 10

Trafo Substation(21)-> 11

Trafo Substation(23)-> 12

Trafo Substation(25)-> 13

Trafo Substation(26)-> 14

Trafo Substation(27)-> 15

Trafo Substation(28)-> 16

Trafo Substation(29)-> 17


In [ ]:

incidence_mat = np.identity(1 + 5 + 17) * -1

# 20 kV substations 
incidence_mat[1 + 1 - 1,1 - 1] = 1 #1st 20 kV substation, gets the electricity from root
incidence_mat[1 + 2 - 1,1 - 1] = 1 #2nd 20 kV substation, gets the electricity from root
incidence_mat[1 + 3 - 1,1 - 1] = 1 #3rd 20 kV substation, gets the electricity from root
incidence_mat[1 + 4 - 1,1 - 1] = 1 #4th 20 kV substation, gets the electricity from root
incidence_mat[1 + 5 - 1,1 - 1] = 1 #5th 20 kV substation, gets the electricity from root


# Trafo substations
incidence_mat[1 + 5 + 1 - 1,1 + 2 - 1] = 1 #first trafo substation, gets the electricity from 2nd 20 kV subst.
incidence_mat[1 + 5 + 2 - 1,1 + 5 + 4 - 1] = 1 # trafo 4 -> trafo 2
incidence_mat[1 + 5 + 3 - 1,1 + 5 + 17 - 1] = 1 # trafo 29(index 17) -> trafo 3
incidence_mat[1 + 5 + 4 - 1,1 + 1 - 1] = 1 # 20 kV(1) -> trafo 4
incidence_mat[1 + 5 + 5 - 1,1 + 5 + 6 - 1] = 1 # trafo 9(index 6) -> trafo 7(index 5)
incidence_mat[1 + 5 + 6 - 1,1 + 4 - 1] = 1 # 20 kV(4) -> trafo 9(index 6)
incidence_mat[1 + 5 + 7 - 1,1 + 5 + 16 - 1] = 1 # trafo 28(index 16) -> trafo 11(index 7)
incidence_mat[1 + 5 + 8 - 1,1 + 5 + 7 - 1] = 1 # trafo 11(index 7) -> trafo 14(index 8)
incidence_mat[1 + 5 + 9 - 1,1 + 3 - 1] = 1 # 20 kV(3) -> trafo 16(index 9)
incidence_mat[1 + 5 + 10 - 1,1 + 5 + 5 - 1] = 1 # trafo 7(index 5) -> trafo 18(index 10)
incidence_mat[1 + 5 + 11 - 1,1 + 5 + 1 - 1] = 1 # trafo 1(index 1) -> trafo 21(index 11)
incidence_mat[1 + 5 + 12 - 1,1 + 5 + 4 - 1] = 1 # trafo 4(index 4) -> trafo 23(index 12)
incidence_mat[1 + 5 + 13 - 1,1 + 5 + 2 - 1] = 1 # trafo 2(index 2) -> trafo 25(index 13)
incidence_mat[1 + 5 + 14 - 1,1 + 5 + 5 - 1] = 1 # trafo 7(index 5) -> trafo 26(index 14)
incidence_mat[1 + 5 + 15 - 1,1 + 5 + 17 - 1] = 1 # trafo 29(index 17) -> trafo 27(index 15)
incidence_mat[1 + 5 + 16 - 1,1 + 5 - 1] = 1 # 20 kV(5) -> trafo 28(index 16)
incidence_mat[1 + 5 + 17 - 1,1 + 3 - 1] = 1 # 20 kV(3) -> trafo 29(index 17)

np.set_printoptions(threshold=np.nan)
print(incidence_mat)
np.set_printoptions(threshold=5)

We can understand whether this network is radial or not by checking the rows of this matrix. If there are 2 or more 1 s in a row, it means that there are 2 source nodes for the corresponding node. From this, we conclude that there are multiple paths to the same node, so the radiality is violated. Also if there is a closed loop, there has to be a 2 source nodes for a node. Only one exception to this is i.e 1(root)->2->3->1 , root node has an incoming node. In our case, this is impossible. So, whenever there is a closed loop in the network, there are at least 2 incoming nodes for one node.

As an algorithm, we may find the row sums and if the sum is larger than 0(1 for source node, -1 for destination node) then there are 2 incoming nodes for that node. So the maximum value for the row sums should be 0.

However, to understand the if the the problem with radiality or a closed loop, one has to investigate further, since this algorithm only checks if it is radial or not, does not give a clue about closed loops.

In [ ]:
is_radial =  incidence_mat.sum(1).max() == 0.0
print(is_radial)

## Exercise 4

## Useful Functions Provided

## Radiality Check

In [6]:
def radiality_check(feeders):
  radiality_is_met = True
  for feeder in feeders:
      is_radial = feeder.ciRadial
      if is_radial == 0:
          return (False, 'Grid is meshed')
  return(True, None)

In [7]:
## just to check if the normal 
radiality_check(feeders)[0]   

True

## Feeded Check

In [8]:
def feeded_check(subs): 
  all_are_fed = True
  for sub in subs:
      bus_bar = sub.GetAttribute('e:pBusbar')
      is_energized = bus_bar.IsEnergized()
      if is_energized == 0:
          return (False, sub.loc_name +' Is not energized')
  return(True, None)

In [9]:
feeded_check(subs)[0]

True

## Calculate Losses

In [10]:
    def total_losses(lines):   
        losses = 0
        for line in lines:
            losses += line.GetAttribute('c:Losses')
        return(losses)

In [11]:
total_losses(lines)

15.888602217698068

## To-Do: Find different scenarios

In [17]:
scenarios = {'firstScenario' : ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)','Line(9)']}
print(feeded_check(subs))
for sce_key in scenarios.keys():
    for line in lines:
        if  not line.loc_name in scenarios[sce_key ]:
            line.SwitchOn()
        else:
            line.SwitchOff()
    app.GetFromStudyCase('ComLdf').Execute()
    lines=app.GetCalcRelevantObjects('*.ElmLne')
    print(total_losses(lines))
    
    

(True, None)
14.364221207753435


## Graphic Save

This saves a graphic of grid and geolocation visualization. The name is given as input(figname) and the resulting images are under graphics/Exercise/RES

In [ ]:
project_folder_in_PF = app.GetProjectFolder('netmod')
graphic_board_folder = project_folder_in_PF.GetContents()[0].GetContents()


grid = [i for i in graphic_board_folder if 'Grid' in i.loc_name][0]
geo = [i for i in graphic_board_folder if 'Geographical' in i.loc_name][0]


def get_graphic(figname, boards=[grid,geo]):

    def project_new_folder(graph_path, grid_name,date_and_time,figname):
        new_path = graph_path+'/ '+grid_name
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        graphic_object = app.GetFromStudyCase('SetDesktop')
        filename = os.path.join(new_path, date_and_time)
        res = graphic_object.WriteWMF(filename)
        img = PythonMagick.Image()
        img.density("125")
        img.read(filename + ".wmf")
        img.write(os.path.join(new_path, figname + "-"+ date_and_time + ".png")) # or .jpg

    #graph_path = r'C:\Users\.........'
    graph_path = r'graphics/Exercise/RES'
    project_name = app.GetActiveProject().loc_name
    new_path = graph_path +'/ '+ project_name
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    for key in boards:
        key.Show()
        this_graphic = key.loc_name
        graph_name = key.loc_name
        sim_dt_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
        usable_time = sim_dt_time.replace(':', '.')
        project_new_folder(new_path, graph_name , usable_time, figname)

In [ ]:
get_graphic("aha")